In [71]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.stats import norm

In [101]:
stocks = ["DLS", "GEM", "TLTD"]
start_date = "2020-01-02"
end_date = "2024-04-12"

data=yf.download(stocks,start=start_date,end=end_date)['Close']
returns=data.pct_change().dropna()
weights=np.array([.23,.18,.59])#manually putting the weights, can change later for github

expected_returns = returns.mean()
stdev = returns.std()
variance = stdev ** 2

cov_matrix = returns.cov()
corr_matrix = returns.corr()

[*********************100%***********************]  3 of 3 completed


In [103]:
def portfolio_performance(weights, mean_returns, cov_matrix):
    port_return = np.dot(weights, mean_returns)
    port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return port_return, port_volatility

ret , vol = portfolio_performance(weights,expected_returns,cov_matrix)
print("Daily Portfolio Return:", ret)
print("Daily Portfolio Volatility:", vol)

Daily Portfolio Return: 0.000208169249209065
Daily Portfolio Volatility: 0.012531824055580445


In [105]:
def historical_var(returns, confidence_level):
    return -np.percentile(returns, (1 - confidence_level) * 100)

def delta_normal_var(port_return, port_volatility, confidence_level):
    z_score = norm.ppf(1 - confidence_level)
    return - (port_return + z_score * port_volatility)

def monte_carlo_var(weights, mean_returns, cov_matrix, confidence_level, simulations=100000):
    np.random.seed(42)
    mean = np.dot(weights, mean_returns)
    std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    simulated_returns = np.random.normal(mean, std_dev, simulations)
    return -np.percentile(simulated_returns, (1 - confidence_level) * 100)

confidence_levels = [0.90, 0.95, 0.99]

Historical_VaR = {cl: historical_var(returns @ weights, cl) for cl in confidence_levels}
Delta_VaR = {cl: delta_normal_var(ret, vol, cl) for cl in confidence_levels}
Montecarlo_VaR = {cl: monte_carlo_var(weights, expected_returns, cov_matrix, cl) for cl in confidence_levels}

print("VaR Estimates")
for cl in confidence_levels:
    print(f"\nConfidence Level: {cl*100}%")
    print(f"Historical VaR: {Historical_VaR[cl]:.6f}")
    print(f"Delta-normal VaR: {Delta_VaR[cl]:.6f}")
    print(f"Monte Carlo VaR: {Montecarlo_VaR[cl]:.6f}")


VaR Estimates

Confidence Level: 90.0%
Historical VaR: 0.012653
Delta-normal VaR: 0.015852
Monte Carlo VaR: 0.015849

Confidence Level: 95.0%
Historical VaR: 0.016877
Delta-normal VaR: 0.020405
Monte Carlo VaR: 0.020388

Confidence Level: 99.0%
Historical VaR: 0.033226
Delta-normal VaR: 0.028945
Monte Carlo VaR: 0.029049
